In [34]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, pairing, Z1, Z2
import galois
GF = galois.GF(curve_order)

In [36]:
import numpy as np

O = np.array([
[0,0,0,1,0,0,0,0],
[0,0,0,0,1,0,0,0],
[0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0],
[0,0,0,0,0,0,0,1],
[0,1,0,0,0,0,0,0]
])
L = np.array([
[ 0,0,1,0,0,0,0,0],
[ 0,0,1,0,0,0,0,0],
[ 3,0,0,0,0,0,0,0],
[ 5,0,0,0,0,0,0,0],
[10,0,0,0,0,0,0,0],
[ 3,0,0,0,0,1,1,1]
])
R = np.array([
[0,0,1,0,0,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,0,0,1,0,0,0],
[0,0,0,1,0,0,0,0],
[0,0,1,0,0,0,0,0],
[1,0,0,0,0,0,0,0]
])

a=[1, 553, 5, 25, 125, 375, 125, 50] # pour etre comme dans le papier
result = O.dot(a) == np.multiply(L.dot(a),R.dot(a))
a=GF(a)


assert result.all(), "Le produit ne fonctionne pas"
print("Lw*Rw = Ow : OK")

def interpolate_lagrange(col, GF, num_eq):
    xs = GF(np.array([i for i in range(1, num_eq+1)]))
    poly = galois.lagrange_poly(xs, col)
    poly_return = np.pad(poly.coeffs, (max(num_eq - len(poly.coeffs), 0), 0), 'constant') #To have vectors of the same size
    return poly_return

num_eq = len(L)

p=curve_order

L_galois = GF(np.array(L) % p)
R_galois = GF(np.array(R) % p)
O_galois = GF(np.array(O) % p)

U = []
V = []
W = []

for i in range(len(a)):
    U.append(interpolate_lagrange(L_galois[:, i], GF, num_eq))
    V.append(interpolate_lagrange(R_galois[:, i], GF, num_eq))
    W.append(interpolate_lagrange(O_galois[:, i], GF, num_eq))

U=GF(U)
V=GF(V)
W=GF(W)

# print("Matrix U:")
# for line in U:
#     for elem in line:
#         print(elem, end=" ")
#     print()

# print("\nMatrix V:")
# for line in V:
#     for elem in line:
#         print(elem, end=" ")
#     print()

# print("\nMatrix W:")
# for line in W:
#     for elem in line:
#         print(elem, end=" ")
#     print()

Lw*Rw = Ow : OK


In [37]:
values = [i for i in range(1, len(U[0])+1)] # Car on a 6 colonnes
poly_t = galois.Poly([1], field=GF)  # Start with the polynomial '1' in the given field
for val in values:
    poly_t *= galois.Poly([1, -val], field=GF)
t=[]
for elem in poly_t.coeffs:
    t.append(int(elem))

Uw = galois.Poly(np.matmul(np.transpose(U), a))
Vw = galois.Poly(np.matmul(np.transpose(V), a))
Ww = galois.Poly(np.matmul(np.transpose(W), a))
t_poly=galois.Poly(t,field=GF)

h = (Uw * Vw - Ww) // t_poly
h_rem = (Uw * Vw - Ww) % t_poly

assert h_rem == 0, "h(x) is not a multiple of t(x)"

In [38]:
print("Uw(x) = ", Uw)
print("Vw(x) = ", Vw)
print("Ww(x) = ", Ww)
print("h(x) = ", h)
print("t(x) = ", t_poly)
print(" ")
res1 = Uw*Vw
res2= Ww+t_poly* h
print("Uw(x) * Vw(x) = ", res1)
print("Ww(x) * t(x) + h(x) = ", res2)

if res1 == res2:
    print("Uw(x) * Vw(x) = Ww(x) * t(x) + h(x) : OK")



Uw(x) =  14774563938491510775016323878048660684770145970280823181996287825938670734546x^5 + 3648040478639879203707734290876212514758060733402672390616367364429301415868x^4 + 10032111316259667810196269299909584415584667016857349074195010252180578894213x^3 + 7296080957279758407415468581752425029516121466805344781232734728858602830837x^2 + 8025689053007734248157015439927667532467733613485879259356008201744463116328x + 21888242871839275222246405745257275088548364400416034343698204186575808495064
Vw(x) =  20429026680383323540763312028906790082645140107054965387451657240804087929235x^5 + 14592161914559516814830937163504850059032242933610689562465469457717205663881x^4 + 7296080957279758407415468581752425029516121466805344781232734728858602830952x^3 + 7296080957279758407415468581752425029516121466805344781232734728858602834704x^2 + 16051378106015468496314030879855335064935467226971758518712016403488926226275x + 1809
Ww(x) =  111265234598516315713085895871724481700120852368781507913799204615093

In [39]:
#Introduction de alpha beta
alpha = GF(2)
beta = GF(3)
res1 = (alpha+Uw)*(beta+Vw) 
res2= alpha*beta
res3 = beta*Uw + alpha*Vw + Ww + h*t_poly

print("alpha = ", alpha)
print("beta = ", beta)
print(" ")
print("(alpha+Uw(x)) * (beta+Vw) = ",res1 )
print("alpha*beta + beta*Uw + alpha*Vw + Ww + h*t_poly = ", res2+res3)
if res1 == res2+res3:
    print("(alpha+Uw(x)) * (beta+Vw) = alpha*beta + beta*Uw + alpha*Vw + Ww + h*t_poly : OK")

alpha =  2
beta =  3
 
(alpha+Uw(x)) * (beta+Vw) =  18568526036276985146872367540559921700118529133019602468237309884945144207081x^10 + 12464138302019587279334758827160392758756707505792464001272588495133446505579x^9 + 11187324134495629558037051825353718378591386249101528664556859917583190992486x^8 + 3526439129351883230250809814513672097599458708955916644262488452281658171725x^7 + 19346774671720159376996684189280180369933582089478839244902134922690061123801x^6 + 1520016866099949668211555954531755214482525305584446829423486401845544757034x^5 + 8876898498023706062355486774465450452577947784613169483833160586777960987434x^4 + 18969810488927371859280218312556305076741915813693896431205110295032376145749x^3 + 7685205275001345522477626906112554364423647945034963169565147247731053337273x^2 + 7296080957279758407415468581752425029516121466805344781232734728858607248391x + 21888242871839275222246405745257275088548364400416034343698204186575807497205
alpha*beta + beta*Uw + alpha*Vw + Ww + h*t_pol

In [44]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, pairing, Z1

alpha_G1=multiply(G1, int(alpha))
beta_G2=multiply(G2, int(beta))
beta_G1=multiply(G1, int(beta))

x = 7
Uw_x = Uw(x)
Vw_x = Vw(x)
Ww_x = Ww(x)

A_1=multiply(G1, int(Uw_x))
B_1=multiply(G1, int(Vw_x))
B_2=multiply(G2, int(Vw_x))
C_1=multiply(G1, int(Ww_x))

res1=add(alpha_G1, A_1)
res2=add(beta_G2, B_2)
res3=multiply(G2, int(alpha)*int(beta))
temp = beta*Uw_x + alpha*Vw_x + Ww_x + h(x)*t_poly(x)
res4=multiply(G1, int(temp))

pairing1=pairing(res2, res1)
pairing2=pairing(beta_G2,alpha_G1)

pairing3=pairing(G2, res4)
print(pairing1==pairing2*pairing3) #True

#CALCULE AVEC LE MAX DE DEVELOPPEMENT
pairing3a=pairing(G2, multiply(A_1, int(beta)))
pairing3b=pairing(G2, multiply(B_1, int(alpha)))
pairing3c=pairing(G2, C_1)
pairing3d=pairing(G2, multiply(G1, int(h(x)*t_poly(x))))

print(pairing3a*pairing3b*pairing3c*pairing3d==pairing3) #True


True
True
